In [10]:
%load_ext sparksql_magic
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col

spark = SparkSession.builder.appName("CreateTable").getOrCreate()

The sparksql_magic extension is already loaded. To reload it, use:
  %reload_ext sparksql_magic


25/03/17 06:43:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## 🌊🐳🧊 Example of creating an Iceberg Table

In [2]:
%%sql
CREATE TABLE
  IF NOT EXISTS iceberg.dim_repo (id BIGINT, name STRING, url STRING) USING ICEBERG

25/03/17 06:10:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [3]:
%%sql
CREATE TABLE
  IF NOT EXISTS iceberg.dim_org (id BIGINT, name STRING, url STRING) USING ICEBERG

++
||
++
++

## event tables TBD

In [4]:
%%sql
CREATE TABLE
  IF NOT EXISTS iceberg.fct_issue_events (
    event_id BIGINT, -- Unique event identifier (PK)
    action STRING, -- Type of action (opened, closed, reopened)
    issue_id BIGINT, -- FK to Issues dimension
    user_id BIGINT, -- FK to Users dimension
    org_id BIGINT, -- FK to Organizations dimension
    created_at TIMESTAMP -- When the event occurred
  ) USING ICEBERG PARTITIONED BY (action);

++
||
++
++

## dim issues should store data related to issues that is immutable

In [5]:
%%sql
CREATE TABLE IF NOT EXISTS iceberg.fct_issue_event (
  -- Primary Key & Event Type
  event_id BIGINT COMMENT 'Unique identifier for the event',
  action STRING COMMENT 'Type of issue event (created, closed, reopened)',

  -- Issue Metadata
  issue_title STRING COMMENT 'Title of the issue',
  issue_body STRING COMMENT 'Detailed description of the issue',
  author_association STRING COMMENT 'Association of the issue creator with the repository',
  issue_state STRING COMMENT 'Current state of the issue (open, closed)',
  issue_state_reason STRING COMMENT 'Reason why the issue was closed, if applicable',

  -- Issue Creator (STRUCT)
  issue_creator STRUCT<
    id: BIGINT COMMENT 'Unique user ID',
    login: STRING COMMENT 'GitHub username of the issue creator',
    url: STRING COMMENT 'API URL of the user',
    is_bot: INT COMMENT '1 if the user is a bot, 0 otherwise'
  > COMMENT 'Details of the issue creator',

  -- Other Issue Attributes
  issue_comment_count BIGINT COMMENT 'Total number of comments on the issue',
  num_assignees INT COMMENT 'Number of assignees for the issue',

  -- Labels (Stored as an Array of Structs)
  labels ARRAY<STRUCT<
    color: STRING COMMENT 'Hex color of the label',
    default: BOOLEAN COMMENT 'TRUE if this is a default GitHub label',
    description: STRING COMMENT 'Label description',
    id: BIGINT COMMENT 'Unique label ID',
    name: STRING COMMENT 'Label name',
    node_id: STRING COMMENT 'GraphQL node ID',
    url: STRING COMMENT 'API URL for the label'
  >> COMMENT 'Labels assigned to the issue',

  -- Milestone (STRUCT)
  milestone STRUCT<
    id: BIGINT COMMENT 'Unique milestone ID',
    title: STRING COMMENT 'Milestone title',
    description: STRING COMMENT 'Milestone description',
    state: STRING COMMENT 'State of the milestone (open, closed)',
    closed_issues: BIGINT COMMENT 'Number of closed issues in the milestone',
    open_issues: BIGINT COMMENT 'Number of open issues in the milestone',
    due_on: TIMESTAMP COMMENT 'Milestone deadline (if available)',
    created_at: TIMESTAMP COMMENT 'Timestamp when milestone was created',
    updated_at: TIMESTAMP COMMENT 'Timestamp when milestone was last updated',
    closed_at: TIMESTAMP COMMENT 'Timestamp when milestone was closed'
  > COMMENT 'Details of the milestone associated with the issue',

  -- Timestamps (Time-Based Features)
  issue_created_at TIMESTAMP COMMENT 'Timestamp when the issue was created',
  issue_closed_at TIMESTAMP COMMENT 'Timestamp when the issue was closed',
  issue_updated_at TIMESTAMP COMMENT 'Timestamp when the issue was last updated',
  event_created_at TIMESTAMP COMMENT 'Timestamp when the event occurred (not real event time)',

  -- Foreign Keys (For Joins)
  issue_id BIGINT COMMENT 'Foreign key to Issues table',
  user_id BIGINT COMMENT 'Foreign key to Users table (actor.id)',
  org_id BIGINT COMMENT 'Foreign key to Organizations table'
)
USING ICEBERG
PARTITIONED BY (action)
COMMENT 'Fact table for GitHub issue event tracking';

++
||
++
++

In [11]:
%%sql 
SELECT
  (SUM(file_size_in_bytes) / (1024 * 1024 * 1024)) * 12 AS table_size_gb
FROM
  iceberg.fct_issue_event.files
  -- iceburg.fct_issue_event

25/03/17 06:43:38 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


table_size_gb
9.669650360941887


In [7]:
#%%sql 
#DROP TABLE iceberg.fct_issue_event;


In [8]:
%%sql 
SELECT
  (SUM(file_size_in_bytes) / (1024 * 1024 * 1024)) * 12 AS table_size_gb
FROM
  iceberg.dim_repo.files

table_size_gb
None
